## Data preparation

### Raw data

In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import visuals as vs
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

df = pd.read_csv('.\\data\\orderWithProfit.csv', header=0)
# filtered_df = df[df['orderdate'].isnull()]
df = df.dropna()
df["orderDate"] = df["orderDate"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df["takenDate"] = df["takenDate"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df["shipDate"] = df["shipDate"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df["transitDuration"] = (df["shipDate"]-df["takenDate"])/ np.timedelta64(1, 's')
df["fulfillDuration"] = (df["shipDate"]-df["orderDate"])/ np.timedelta64(1, 's')

df["amount"] = df["red"]+df["blue"]+df["yellow"]+df["black"]+df["white"]

### Integrate User data

In [3]:
dic = {}

# Combine with customer info
df_tmp = pd.read_csv('.\\data\\orderWithCustomer.csv', header=0)
df = pd.merge(df, df_tmp, how='inner', left_on="customer", right_on="name",suffixes=('_x', '_y'),)

for key in ["customer", "name","age", "sex", "city", "state", "country",\
                 "income", "credit","education", "occupation","orderCount","totalProfit"]:
    dic[key] = {}
    ## Add Customer ID (Integer number)
    id = 1
    for _,name in df[[key]].drop_duplicates()[key].iteritems():
        dic[key][name] = id # id starts from 0
        id = id+1
    df[key] = df[key].apply(lambda x: dic[key][x])

    
print "dictionary keys:",dic.keys()
print df.info()

SyntaxError: invalid syntax (<ipython-input-3-17bc5bf11c7c>, line 17)

### Features

We divide all the features into the following subgroups. Each of the subgroup has its unique source of data. For example. the dataset of "user" are derived from the user information and their order records in our system. For each of the subgroup, we have specific target that we want to explore the pattern of. In "users", we want to analyze the relationship between the profit of an order and the information of the user who makes the order. In "Order", we want to analyze the relationship between the profit of an order and the number of different inventories. In "DUration", we want to analyze the relationship between the time spent on transiting and the running data of the robots. In the last dataset, we combine several typical features from each of the dataset and put into a single dataset to see if we can dig out something interesting.

#### User
customer_id              
name            
age       
sex       
city         
state            
country          
income             
credit              
education    
occupation        
orderCount       
totalProfit   

#### Order
customer_id

green,blue,black,yellow,red,white

**amount** = green + blue + black + yellow + red + white

#### Duration

orderDate,takenDate,shipDate

**transitDuration** = shipDate - takenDate

**fufillDuration** = shipDate - orderDate

shipVisitCount

#### Cost, revenue, profit
productSales,shipHandleCost

**totalRevenue**=productSales + shipHandleCost

productCOGS, *orderProcessCost(=10)*, shipVisitCost

**totalCost** = productCOGS + *orderProcessCost* + shipVisitcCost

**profit**= **totalRevenue** - **totalCost**

### PCA

In [ ]:
X = {}
index = {}
target = {"users":"profit","order":"profit","duration":"transitDuration","comples":"profit"}

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
index["users"]=["age","sex", "city", "state","country","income","credit","education","occupation","orderCount","totalProfit","profit"]


scaler = StandardScaler()

X["users"] = scaler.fit_transform(df[index["users"]].values)
pca = PCA(n_components=2)
X["users"] = pca.fit_transform(X["users"])

pca_results = vs.pca_results(df[index["users"]], pca)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
index["colors"]=["green","blue", "black", "yellow","red","white","profit"]

scaler = StandardScaler()

X["colors"] = scaler.fit_transform(df[index["colors"]].values)
pca = PCA(n_components=2)
X["colors"] = pca.fit_transform(X["colors"])

pca_results = vs.pca_results(df[index["colors"]], pca)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
index["duration"]=["transitDuration","fulfillDuration","shipVisitCount", "amount", "profit"]

scaler = StandardScaler()

X["duration"] = scaler.fit_transform(df[index["duration"]].values)

pca = PCA(n_components=2)
X["duration"] = pca.fit_transform(X["duration"])

pca_results = vs.pca_results(df[index["duration"]], pca)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
index["price"]=["amount","transitDuration", "shipVisitCount", "totalRevenue","totalCost","profit"]

scaler = StandardScaler()

X["price"] = scaler.fit_transform(df[index["price"]].values)
pca = PCA(n_components=2)
X["price"] = pca.fit_transform(X["price"])

pca_results = vs.pca_results(df[index["price"]], pca)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
index["complex"]=["age","sex", "income", "credit",\
        "totalProfit","amount","transitDuration",\
        "profit" ]

scaler = StandardScaler()

X["complex"] = scaler.fit_transform(df[index["complex"]].values)

pca = PCA(n_components=2)
X["complex"] = pca.fit_transform(X["complex"])

pca_results = vs.pca_results(df[index["complex"]], pca)

### Data Selection
Based on the result of PCA on each of the dataset, we run on the K-means algorithm with cluster number 8. Then, we compare the outcome and choose the best one to perform further analysis. (As our data is limited and noisy, we will not continue analyzing the rest as the result is not reliable enough. )

We use two metrics to measure the quality of outcome. One is SSE(Sum of Squared Distance), and another is Calinski and Harabaz score, which is also a internal cluster criterion. The reason we apply both metrics is we observed that the SSE keeps decreasing if K increase, which might be a sign of overfitting. Thus, we introduce this metric to improve the confidendce on K.

In [ ]:
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabaz_score
import matplotlib.pyplot as plt
import numpy as np

sse = {}
chs = {}
fig = plt.figure(1, figsize=(10,20))

X_keys = X.keys()

for i in xrange(4):
    #KMeans    
    km = KMeans(n_clusters=8,
               max_iter=1000)

    y_pred = km.fit_predict(X[X_keys[i]])
    
    #Measuremenet: SSE
    sse[i] = km.inertia_
    #Measuremenet: Calinski and Harabaz score
    chs[i]= calinski_harabaz_score(X[X_keys[i]], y_pred)

#Plotting SSE
fig = plt.figure(2,figsize=(10,3))
plt.subplot(1,2,1)
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Dataset#")
plt.ylabel("SSE")

plt.subplot(1,2,2)
plt.plot(list(chs.keys()), list(chs.values()))
plt.xlabel("Dataset#")
plt.ylabel("C&H Score")

# Chooose the dataset index
i_sse = min(sse.items(), key = lambda x:x[1])[0]
i_chs = max(chs.items(), key = lambda x:x[1])[0]

INDEX = X_keys[i_chs]
# INDEX = "users"

if i_sse == i_chs:
    print "SSH and C&H scores agree on usng dataset ",INDEX
else:
    print "SSH and C&H scores conflict, choose C&H result: dataset ",INDEX

### PCA Analysis

## K-Means

### SSE change with K

In [ ]:
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabaz_score
import matplotlib.pyplot as plt
import numpy as np

data = X[INDEX]

sse = {}
chs = {}
fig = plt.figure(1, figsize=(20,40))
for k in xrange(1,11):
    #KMeans    
    km = KMeans(n_clusters=k,
               max_iter=1000)

    y_pred = km.fit_predict(data)
    sse[k] = km.inertia_

    #Measuremenet: Calinski and Harabaz score
    if k!=1:
        chs[k]= calinski_harabaz_score(data, y_pred)
    
    plt.subplot(5,2,k)
    plt.scatter(data[:, 0], data[:, 1],  s=30, c=y_pred, cmap='viridis')
    plt.title("K Means (K=%d)"%k, fontsize=14)

#Plotting SSE
fig = plt.figure(2,figsize=(10,3))
plt.subplot(1,2,1)
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")

plt.subplot(1,2,2)
plt.plot(list(chs.keys()), list(chs.values()))
plt.xlabel("Number of cluster")
plt.ylabel("C&H Score")

# Chooose the best K
K = max(chs.items(), key = lambda x:x[1])[0]

### Cluster with best K

In [ ]:
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabaz_score
import matplotlib.pyplot as plt
import numpy as np

#KMeans    
km = KMeans(n_clusters= K,
           max_iter=2000)

y_pred = km.fit_predict(data)

#Measuremenet: Calinski and Harabaz score
print "Calinski & Harabaz score:",calinski_harabaz_score(data, y_pred)  
print "SSE:",km.inertia_ 

#Plotting
fig = plt.figure(1, figsize=(10,10))
plt.scatter(data[:, 0], data[:, 1], c=y_pred, s=30, cmap='viridis')
plt.title("K Means (K=%d)"%K, fontsize=14)    
plt.xlabel("PC1")
plt.ylabel("PC2")

### Examine clustered data
We plotted the clustered data by groups with the following features:
    * Amount and visitCount
Amount of items in the order and the number of robots that the order is split to.
    * Customer
Customers who had order records on the system.
    * Gender
Gender of the cusotmer who made the order. We plotted both features with duration and their combination. 
    * Education and occupation
Education and occupation of the cusotmer who made the order. We plotted both features with duration and their combination. 
    * Location
Location of the cusotmer who made the order. We plotted both features with duration and their combination. 


#### Amount, visitCount

According to the plottings below, it can be told that the transit duration has strong relationship with the amount of items in the order. 
* K = 4,7 shows the range of transit duration locates around 50-100 if the amount is small (less than 20). 
* K = 2,6,8,9 shows the range of duration is around 100-175 when amount is medium (20 - 30). 
* K = 3, 5 shows the range of duration is around 150-300 when amount is large (larger than 30). 

In [ ]:
cluster = {}
fig, axes = plt.subplots(K/2+K%2,2,sharex=True, sharey=True,figsize=(10,20))
for k in xrange(K):
    cluster[k] = df.loc[[i for i in range(len(km.labels_)) if km.labels_[i]==k]]
    ax = axes[k/2, k%2]
    ax.scatter(cluster[k][target[INDEX]],cluster[k]["amount"])
    ax.set_xlabel(target[INDEX])
    ax.set_ylabel('amount')
    ax.set_title('K=%d'%k)
    if(k==K-1):
        break;

In [ ]:
cluster = {}
fig, axes = plt.subplots(K/2+K%2,2,sharex=True, sharey=True,figsize=(10,20))
for k in xrange(K):
    cluster[k] = df.loc[[i for i in range(len(km.labels_)) if km.labels_[i]==k]]
    ax = axes[k/2, k%2]
    ax.scatter(cluster[k][target[INDEX]],cluster[k]["shipVisitCount"])
    ax.set_xlabel(target[INDEX])
    ax.set_ylabel('shipVisitCount')
    ax.set_title('K=%d'%k)
    if(k==K-1):
        break;

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

for k in xrange(K):
    fig = plt.figure(1, figsize=(5,5))
    ax = Axes3D(fig, rect=[0, 0, 0.95, 1])
    ax.scatter(cluster[k]["amount"], cluster[k]["shipVisitCount"], cluster[k][target[INDEX]],edgecolor="k", s=100)
    ax.set_xlabel("amount")
    ax.set_ylabel("shipVisitCount")
    ax.set_zlabel(target[INDEX])
    plt.title("K=%d"%k, fontsize=14)
    plt.tight_layout()
    plt.show()


#### Customer 
In this part, we examine whether transit duration changes according to the specific user. 
And from the plotting of K = 5, it can be told that the extreme long transit time tend happen to some users whose ID nubmer is small.

In [ ]:
cluster = {}
fig, axes = plt.subplots(K/2+K%2,2,sharex=True, sharey=True,figsize=(10,20))
for k in xrange(K):
    cluster[k] = df.loc[[i for i in range(len(km.labels_)) if km.labels_[i]==k]]
    ax = axes[k/2, k%2]
    ax.scatter(cluster[k][target[INDEX]],cluster[k]["customer"])
    ax.set_xlabel(target[INDEX])
    ax.set_ylabel('customer')
    ax.set_title('K=%d'%k)
    if(k==K-1):
        break;

#### Gender
From plotting of gender (female = 1, male = 2), it can be told that males in our case tend to wait longer for their items. Also, extreme long durations (more than 200) tend to happen to those males.

In [ ]:
cluster = {}
fig, axes = plt.subplots(K/2+K%2,2,sharex=True, sharey=True,figsize=(10,20))


for k in xrange(K):
    cluster[k] = df.loc[[i for i in range(len(km.labels_)) if km.labels_[i]==k]]
    ax = axes[k/2, k%2]
    ax.scatter(cluster[k][target[INDEX]],cluster[k]["sex"])
    ax.set_xlabel(target[INDEX])
    ax.set_ylabel('sex')
    ax.set_title('K=%d'%k)
    if(k==K-1):
        break;

#### Location of the cusotmer

We also tried to dig out the relationship between the location of the user and the time takes for transition of order. It turns out that there is no specific pattern between those two factors.

In [ ]:
cluster = {}
fig, axes = plt.subplots(K/2+K%2,2,sharex=True, sharey=True,figsize=(10,20))


for k in xrange(K):
    cluster[k] = df.loc[[i for i in range(len(km.labels_)) if km.labels_[i]==k]]
    ax = axes[k/2, k%2]
    ax.scatter(cluster[k][target[INDEX]],cluster[k]["city"])
#     ax.set_xlim([1, id+1])
    ax.set_xlabel(target[INDEX])
    ax.set_ylabel('city')
    ax.set_title('K=%d'%k)
    if(k==K-1):
        break;

In [ ]:
cluster = {}
fig, axes = plt.subplots(K/2+K%2,2,sharex=True, sharey=True,figsize=(10,20))


for k in xrange(K):
    cluster[k] = df.loc[[i for i in range(len(km.labels_)) if km.labels_[i]==k]]
    ax = axes[k/2, k%2]
    ax.scatter(cluster[k][target[INDEX]],cluster[k]["state"])
#     ax.set_xlim([1, id+1])
    ax.set_xlabel(target[INDEX])
    ax.set_ylabel('state')
    ax.set_title('K=%d'%k)
    if(k==K-1):
        break;

### Setup for HCA

In [ ]:
import matplotlib.pyplot as plt
from sklearn import manifold
import time as time
from sklearn.cluster import AgglomerativeClustering
from sklearn import preprocessing
from scipy import ndimage
import mpl_toolkits.mplot3d.axes3d as p3
import seaborn as sns
from scipy.spatial import distance_matrix
from scipy.cluster.hierarchy import dendrogram, linkage

def showdata(X):
    # #############################################################################
    # Compute clustering
    print("Compute unstructured hierarchical clustering...")
    
    st = time.time()
    ward = AgglomerativeClustering(n_clusters=6, linkage='ward').fit(X)
    elapsed_time = time.time() - st
    label = ward.labels_
    print("Elapsed time: %.2fs" % elapsed_time)
    print("Number of points: %i" % label.size)
    
    # #############################################################################
    # Plot result
    fig = plt.figure()
    ax = p3.Axes3D(fig)
    ax.view_init(7, -80)
    for l in np.unique(label):
        ax.scatter(X[label == l, 0], X[label == l, 1], X[label == l, 2],
                   color=plt.cm.jet(np.float(l) / np.max(label + 1)),
                   s=20, edgecolor='k')
    plt.title('Without connectivity constraints (time %.2fs)' % elapsed_time)
    
    # #############################################################################
    # Define the structure A of the data. Here a 10 nearest neighbors
    from sklearn.neighbors import kneighbors_graph
    connectivity = kneighbors_graph(X, n_neighbors=10, include_self=False)
    
    # #############################################################################
    # Compute clustering
    print("Compute structured hierarchical clustering...")
    st = time.time()
    ward = AgglomerativeClustering(n_clusters=6, connectivity=connectivity,
                                   linkage='ward').fit(X)
    elapsed_time = time.time() - st
    label = ward.labels_
    print("Elapsed time: %.2fs" % elapsed_time)
    print("Number of points: %i" % label.size)
    
    # #############################################################################
    # Plot result
    fig = plt.figure()
    ax = p3.Axes3D(fig)
    ax.view_init(7, -80)
    for l in np.unique(label):
        ax.scatter(X[label == l, 0], X[label == l, 1], X[label == l, 2],
                   color=plt.cm.jet(float(l) / np.max(label + 1)),
                   s=20, edgecolor='k')
    plt.title('With connectivity constraints (time %.2fs)' % elapsed_time)
    
    plt.show()

def hca(X, met):
 #   path = sys.argv[1]
    #path = 'data/orderWithCustomer.csv'
    #X, y = read_csv(path)

    # find distance matrix
    d = distance_matrix(X, X)
    for link in ('ward', 'average', 'complete'):
        clustering = AgglomerativeClustering(linkage=link, n_clusters=10).fit(d)
        st = time.time()
        label = clustering.labels_
        elapsed_time = time.time() - st

    # Draw the clustering heatmap based on different methods
    sns.set(color_codes=True)
    g1 = sns.clustermap(X, method='ward', metric=met, figsize=(5, 8))
    g2 = sns.clustermap(X, method='average', metric=met, figsize=(5, 8))
    g3 = sns.clustermap(X, method='complete', metric=met, figsize=(5, 8))

    # Draw the dendrogram of the clustering based on different methods
    plt.figure()
    plt.title('Hierarchical Clustering Dendrogram (ward)')
    plt.xlabel('sample index')
    plt.ylabel('distance')
    Z = linkage(X, 'ward')
    dendrogram(
        Z,
        leaf_rotation=90.,
        leaf_font_size=8.,
    )
    plt.figure()
    plt.title('Hierarchical Clustering Dendrogram (average)')
    plt.xlabel('sample index')
    plt.ylabel('distance')
    Z = linkage(X, 'average')
    dendrogram(
        Z,
        leaf_rotation=90.,
        leaf_font_size=8.,
    )
    plt.figure()
    plt.title('Hierarchical Clustering Dendrogram (complete)')
    plt.xlabel('sample index')
    plt.ylabel('distance')
    Z = linkage(X, 'complete')
    dendrogram(
        Z,
        leaf_rotation=90.,
        leaf_font_size=8.,
    )
    plt.show()

In [ ]:
data = X["users"]
data_scaled = preprocessing.scale(data)

In [ ]:
showdata(data)
showdata(data_scaled)

### HCA Raw Data - Euclidean

In [ ]:

hca(data,'euclidean')

### HCA Scaled Data - Euclidean

In [ ]:
hca(data_scaled,'euclidean')

### HCA Raw Data - Correlation

In [ ]:
hca(data,'correlation')

### HCA Scaled Data - Correlation

In [ ]:
hca(data_scaled,'correlation')